In [6]:
# Import Modules
import sys
sys.path.insert(0, r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\Modules')
import sysconfig; 
from operator import itemgetter
from math import isnan
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.lines as mlines
from scipy.io import loadmat
import os
import Actigraph_Metrics
import tsfel
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import seaborn as sns

In [7]:
# Load Data
os.chdir(r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\PatientData\Patient9')
filename = 'Patient9_BolusDosage.mat'
x_mag = (loadmat(filename)["x_mag"])
dosage = loadmat(filename)["dosage"]

In [8]:
# Generate Peak to peak, mean, auc data for each time point
AUC = []
PEAK_PEAK = []
Mean = []
for i in range(x_mag.shape[0]):
    signal = Actigraph_Metrics.VecMag_MAD(x_mag[i,:],100)
    x = []
    for i in range(len(signal)):
        x.append(i)
    Mean.append(np.mean(signal))
    AUC.append(Actigraph_Metrics.calc_area_under_curve(x,signal))
    PEAK_PEAK.append(max(signal)-min(signal))

In [13]:
# Count Number of Dosage Changes
dosage_change = 0
for i in range(len(dosage[0]) - 1):
    if dosage[0][i] > 0:
        dosage_change += 1
        
print(dosage_change)

10


In [34]:
print(len(dosage[0]) - 1)

mean_threshold = 0.0015
mean_count = 0
peak_threshold = 0.05
peak_count = 0
auc_threshold = 0.05
auc_count = 0

for i in range(len(dosage[0])-1):
    if dosage[0][i + 1] > dosage[0][i]:
        mean_state = abs(Mean[i+1] - Mean[i])
        peak_state = abs(PEAK_PEAK[i+1] - PEAK_PEAK[i])
        auc_state = abs(AUC[i+1] - AUC[i])
        
        if mean_state > mean_threshold:
            mean_count += 1
            
        if peak_state > peak_threshold:
            peak_count += 1
            
        if auc_state > auc_threshold:
            auc_count += 1

print(f"Percentage of dosage changes with Mean is: {100*((mean_count - 1)/dosage_change)}%")
print(f"Percentage of dosage changes with Peak-to-Peak is: {100*((peak_count - 1)/dosage_change)}%")
print(f"Percentage of dosage changes with AUC is: {100*((auc_count - 1)/dosage_change)}%")

37
Percentage of dosage changes with Mean is: 60.0%
Percentage of dosage changes with Peak-to-Peak is: 40.0%
Percentage of dosage changes with AUC is: 70.0%


In [21]:
os.chdir(r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\Modules')
data = pd.read_excel('./Patient_9_Dosage.xlsx', header=2, usecols='A:C')

# Convert the time column to datetime format
data['Time_uniform'] = pd.to_datetime(data['Time_uniform'])

# Plot
plt.figure(figsize=(10, 6))
plt.plot(data['Time_uniform'], data['Midazolam Bolus (mg)'], marker='o', linestyle='-')
plt.title('Drug Dosage Over Time')
plt.xlabel('Time')
plt.ylabel('Drug Dosage')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

C:\Users\sidha\AppData\Local\Temp\ipykernel_35464\1250841259.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time_uniform'] = pd.to_datetime(data['Time_uniform'])


DateParseError: Unknown datetime string format, unable to parse: 1/15/2024  14:00:00 PM, at position 0